# Covid 19 UK Testing


In [1]:
from uk_covid19 import Cov19API
import pandas as pd
import numpy as np
import plotly.express as px
from tsmoothie.smoother import *
from tqdm.notebook import trange, tqdm

In [2]:
filters = ['areaType=nation']

structure = {
    'date': 'date',
    'areaName': 'areaName',
    'areaCode': 'areaCode',
    'newPillarOneTestsByPublishDate': 'newPillarOneTestsByPublishDate',
    'newPillarTwoTestsByPublishDate': 'newPillarTwoTestsByPublishDate',
    'newTestsByPublishDate': 'newTestsByPublishDate',
    'newCasesByPublishDate': 'newCasesByPublishDate'
}

api = Cov19API(filters=filters, structure=structure)
df = api.get_dataframe()

In [3]:
df.sort_values(by=['areaCode','date'], inplace=True)

In [4]:
df['Positive rate'] = df['newCasesByPublishDate'] / df['newTestsByPublishDate'] 

In [5]:
px.line(df, x='date', y = 'Positive rate', color='areaName', width=1280, height=540)

In [6]:
# Full missing values with zero
df['newTestsByPublishDate'] = df['newTestsByPublishDate'].apply(lambda x: x if pd.notnull(x) else 0)
df['newCasesByPublishDate'] = df['newCasesByPublishDate'].apply(lambda x: x if pd.notnull(x) else 0)

my_smoother = LowessSmoother(smooth_fraction=0.05, iterations=1)

df['new cases'] = 0
df['new tests'] = 0
ltlas = df[['areaCode']].groupby(by=['areaCode'], as_index=False).sum().values.tolist()
for ltla in tqdm(ltlas):
    df.loc[df['areaCode'].isin(ltla), ['new tests']] =  my_smoother.smooth(df[df['areaCode'].isin(ltla)]['newTestsByPublishDate']).smooth_data.T    
    df.loc[df['areaCode'].isin(ltla), ['new cases']] =  my_smoother.smooth(df[df['areaCode'].isin(ltla)]['newCasesByPublishDate']).smooth_data.T

In [7]:
df['Smooth positive rate'] = df['new cases'] / df['new tests']
df['Smooth positive rate'] = df['Smooth positive rate'].map(lambda x: None if x > 0.04 else x)
df['Positive rate'] = df['Positive rate'].map(lambda x: None if x > 0.04 else x)

In [8]:
px.line(df, x='date', y = 'Smooth positive rate', color='areaName', width=1280, height=540)

In [9]:
nation = df[(df['areaName'].isin(['England'])) & (df['date'] > '2020-07-01')]
plt = px.line(nation, x='date', y='Smooth positive rate', title='England', width=1280, height=540)
plt.add_trace(px.bar(nation, x='date', y='Positive rate', opacity=0.4, width=1280, height=540).data[0])
plt.show()

In [10]:
plt = px.line(nation, x='date', y='new tests', title='England', width=1280, height=540)
plt.add_trace(px.bar(nation, x='date', y='newTestsByPublishDate', opacity=0.4, width=1280, height=540).data[0])
plt.show()